In [2]:
import os
from pathlib import Path


Path.ls= lambda x: list(x.iterdir())

In [6]:
data_path=Path(r'd:\datasets\data\imagenette2-160')

data_path.ls()

[WindowsPath('d:/datasets/data/imagenette2-160/imagenette2-160.tar'),
 WindowsPath('d:/datasets/data/imagenette2-160/train'),
 WindowsPath('d:/datasets/data/imagenette2-160/val')]

In [11]:
def dir_files(p, fs, extensions=None):
    p=Path(p)
    files=[p/f for f in fs if not f.startswith('.')
          and (not extensions) or f.split('.')[-1].lower() in extension]
    return files

In [14]:
def get_files(root:Path, recurse=False, extensions=None):
    root= Path(root)
    if recurse:
        res=[]
        for p,d,f in os.walk(root):
            d[:] = [o for o in d if not o.startswith('.')]
            res+=dir_files(p, f, extensions)
        return res
    else:
        f=[o.name for o in os.scandir(root) if o.is_file()]
        return dir_files(root, f, extensions)

In [18]:
%time get_files(data_path, recurse=True)

Wall time: 291 ms


[WindowsPath('d:/datasets/data/imagenette2-160/imagenette2-160.tar'),
 WindowsPath('d:/datasets/data/imagenette2-160/train/n01440764/ILSVRC2012_val_00000293.JPEG'),
 WindowsPath('d:/datasets/data/imagenette2-160/train/n01440764/ILSVRC2012_val_00002138.JPEG'),
 WindowsPath('d:/datasets/data/imagenette2-160/train/n01440764/ILSVRC2012_val_00003014.JPEG'),
 WindowsPath('d:/datasets/data/imagenette2-160/train/n01440764/ILSVRC2012_val_00006697.JPEG'),
 WindowsPath('d:/datasets/data/imagenette2-160/train/n01440764/ILSVRC2012_val_00007197.JPEG'),
 WindowsPath('d:/datasets/data/imagenette2-160/train/n01440764/ILSVRC2012_val_00009346.JPEG'),
 WindowsPath('d:/datasets/data/imagenette2-160/train/n01440764/ILSVRC2012_val_00009379.JPEG'),
 WindowsPath('d:/datasets/data/imagenette2-160/train/n01440764/ILSVRC2012_val_00009396.JPEG'),
 WindowsPath('d:/datasets/data/imagenette2-160/train/n01440764/ILSVRC2012_val_00010306.JPEG'),
 WindowsPath('d:/datasets/data/imagenette2-160/train/n01440764/ILSVRC2012_v